# FewShotPromptTemplate

In [2]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.prompts import PromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler()
    ]
)

examples = [
    {
        "question": "What do you know about France?",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """
    },
    {
        "question": "What do you know about Italy?",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """
    },
    {
        "question": "What do you know about Greece?",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """
    },
]

example_prompt = PromptTemplate.from_template("Human: {question}\nAI:{answer}")

prompt = FewShotPromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
    suffix="Human: What do you know about {country}?",
    input_variables=["country"]
)

prompt.format(country="Germany")

'Human: What do you know about France?\nAI:\n        Here is what I know:\n        Capital: Paris\n        Language: French\n        Food: Wine and Cheese\n        Currency: Euro\n        \n\nHuman: What do you know about Italy?\nAI:\n        I know this:\n        Capital: Rome\n        Language: Italian\n        Food: Pizza and Pasta\n        Currency: Euro\n        \n\nHuman: What do you know about Greece?\nAI:\n        I know this:\n        Capital: Athens\n        Language: Greek\n        Food: Souvlaki and Feta Cheese\n        Currency: Euro\n        \n\nHuman: What do you know about Germany?'

# FewShotChatMessagePromptTemplate

In [3]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate # 수정
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import ChatPromptTemplate # 수정

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler()
    ]
)

examples = [
    {
        "country": "France",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "country": "Italy",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "country": "Greece",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
]

# 수정
example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "What do you know about {country}?"),
        ("ai", "{answer}"),
    ]
)

# 수정
example_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

# 수정
final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a geography expert."),
        example_prompt,
        ("human", "What do you know about {country}?"),
    ]
)

chain = final_prompt | chat

chain.invoke({"country": "Korea"})


        There are two Koreas: North Korea and South Korea.
        Capital of South Korea: Seoul
        Capital of North Korea: Pyongyang
        Language: Korean
        Food: Kimchi and Bibimbap
        Currency: South Korea - South Korean Won, North Korea - North Korean Won
        

AIMessageChunk(content='\n        There are two Koreas: North Korea and South Korea.\n        Capital of South Korea: Seoul\n        Capital of North Korea: Pyongyang\n        Language: Korean\n        Food: Kimchi and Bibimbap\n        Currency: South Korea - South Korean Won, North Korea - North Korean Won\n        ')

# LengthBasedExampleSelector

In [7]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import example_selector
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts.prompt import PromptTemplate
from langchain.prompts.example_selector.base import BaseExampleSelector # 추가


chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)


examples = [
    {
        "question": "What do you know about France?",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Italy?",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Greece?",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
]

# 추가
class RandomExampleSelector(BaseExampleSelector):
    def __init__(self, examples):
        self.examples = examples

    def add_example(self, example):
        self.examples.append(example)

    def select_examples(self, input_variables):
        from random import choice

        return [choice(self.examples)]


example_prompt = PromptTemplate.from_template("Human: {question}\nAI:{answer}")

# 추가
example_selector = RandomExampleSelector(
    examples=examples,
)

prompt = FewShotPromptTemplate(
    example_prompt=example_prompt,
    example_selector=example_selector, # 추가
    suffix="Human: What do you know about {country}?",
    input_variables=["country"],
)

prompt.format(country="Brazil")

'Human: What do you know about Italy?\nAI:\n        I know this:\n        Capital: Rome\n        Language: Italian\n        Food: Pizza and Pasta\n        Currency: Euro\n        \n\nHuman: What do you know about Brazil?'

# Serialization and Composition

In [9]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import load_prompt

# JSON 파일에서 prompt 불러오기
prompt = load_prompt("../prompt.json")

# Chat AI 모델 설정
chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)

# 프롬프트에 변수를 적용하여 질문 생성
prompt.format(country="Germany")

'What is the capital of Germany'

In [10]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import load_prompt

# YAML 파일에서 prompt 불러오기
prompt = load_prompt("../prompt.yaml")

# Chat AI 모델 설정
chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)

# 프롬프트에 변수를 적용하여 질문 생성
prompt.format(country="Germany")

'What is the capital of Germany'

In [11]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import PromptTemplate
from langchain.prompts.pipeline import PipelinePromptTemplate

# Chat AI 모델 설정
chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)

# 각각의 프롬프트 부분을 정의합니다.
intro = PromptTemplate.from_template(
    """
    You are a role playing assistant.
    And you are impersonating a {character}
"""
)

example = PromptTemplate.from_template(
    """
    This is an example of how you talk:

    Human: {example_question}
    You: {example_answer}
"""
)

start = PromptTemplate.from_template(
    """
    Start now!

    Human: {question}
    You:
"""
)

# 모든 부분을 하나의 최종 프롬프트로 결합합니다.
final = PromptTemplate.from_template(
    """
    {intro}
                                     
    {example}
                              
    {start}
"""
)

# 각 프롬프트 부분을 파이프라인으로 연결합니다.
prompts = [
    ("intro", intro),
    ("example", example),
    ("start", start),
]

# 파이프라인 프롬프트를 생성합니다.
full_prompt = PipelinePromptTemplate(
    final_prompt=final,
    pipeline_prompts=prompts,
)

# 대화 체인을 생성하고, 파라미터를 넘겨 실행합니다.
chain = full_prompt | chat

chain.invoke(
    {
        "character": "CEO",
        "example_question": "How do you handle stress in the workplace?",
        "example_answer": "It's all about maintaining a balance. I focus on clear communication and prioritize tasks effectively.",
        "question": "What's your approach to leadership?",
    }
)

My approach to leadership is centered around setting a clear vision and direction for the company, while also empowering and supporting my team to achieve their goals. I believe in leading by example, fostering a culture of collaboration and innovation, and always striving for continuous improvement. Communication, transparency, and accountability are key pillars of my leadership style.

AIMessageChunk(content='My approach to leadership is centered around setting a clear vision and direction for the company, while also empowering and supporting my team to achieve their goals. I believe in leading by example, fostering a culture of collaboration and innovation, and always striving for continuous improvement. Communication, transparency, and accountability are key pillars of my leadership style.')

In [12]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import PromptTemplate
from langchain.prompts.pipeline import PipelinePromptTemplate

# Chat AI 모델 설정
chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)

# 각각의 프롬프트 부분을 정의합니다.
intro = PromptTemplate.from_template(
    """
    You are a role playing assistant.
    And you are impersonating a {character}
"""
)

example = PromptTemplate.from_template(
    """
    This is an example of how you talk:

    Human: {example_question}
    You: {example_answer}
"""
)

start = PromptTemplate.from_template(
    """
    Start now!

    Human: {question}
    You:
"""
)

# 모든 부분을 하나의 최종 프롬프트로 결합합니다.
final = PromptTemplate.from_template(
    """
    {intro}
                                     
    {example}
                              
    {start}
"""
)

# 각 프롬프트 부분을 파이프라인으로 연결합니다.
prompts = [
    ("intro", intro),
    ("example", example),
    ("start", start),
]

# 파이프라인 프롬프트를 생성합니다.
full_prompt = PipelinePromptTemplate(
    final_prompt=final,
    pipeline_prompts=prompts,
)

# 대화 체인을 생성하고, 파라미터를 넘겨 실행합니다.
chain = full_prompt | chat
chain.invoke(
    {
        "character": "Startup CEO",
        "example_question": "How do you handle challenges in a startup environment?",
        "example_answer": "I embrace challenges as opportunities for growth and innovation.",
        "question": "What's your approach to leadership in a startup?",
    }
)

I believe in leading by example and empowering my team to take ownership of their work. I prioritize open communication, collaboration, and fostering a culture of creativity and innovation. It's important for me to provide guidance and support while also allowing room for autonomy and growth within the team.

AIMessageChunk(content="I believe in leading by example and empowering my team to take ownership of their work. I prioritize open communication, collaboration, and fostering a culture of creativity and innovation. It's important for me to provide guidance and support while also allowing room for autonomy and growth within the team.")

# Caching

In [13]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.globals import set_llm_cache
from langchain.cache import InMemoryCache

# 캐시를 설정합니다.
set_llm_cache(InMemoryCache())

# chat AI 모델을 설정합니다.
chat = ChatOpenAI(
    temperature=0.1,
    # streaming 및 callback 기능은 비활성화합니다.
    # callbacks=[
    #     StreamingStdOutCallbackHandler(),
    # ],
)

# 첫 번째 예측을 실행합니다. 여기서는 이탈리안 파스타 만드는 법에 대한 답변을 받습니다.
chat.predict("How do you make italian pasta")

'To make Italian pasta, you will need the following ingredients:\n\n- 2 cups of all-purpose flour\n- 2 large eggs\n- 1/2 teaspoon of salt\n- Water (if needed)\n\nHere is a step-by-step guide on how to make Italian pasta:\n\n1. On a clean work surface, pour the flour and make a well in the center.\n2. Crack the eggs into the well and add the salt.\n3. Using a fork, gradually mix the eggs into the flour until a dough forms.\n4. Knead the dough for about 10 minutes until it becomes smooth and elastic. If the dough is too dry, add a little water. If it is too wet, add a little more flour.\n5. Wrap the dough in plastic wrap and let it rest for at least 30 minutes.\n6. After resting, divide the dough into smaller portions and roll each portion out into a thin sheet using a pasta machine or a rolling pin.\n7. Cut the pasta sheet into your desired shape, such as fettuccine or spaghetti.\n8. Cook the pasta in a large pot of boiling salted water for about 2-3 minutes or until al dente.\n9. Drain

In [14]:
# 동일한 질문을 다시 실행합니다. 이번에는 캐시된 답변을 사용하기 때문에 빠르게 응답을 받습니다.
chat.predict("How do you make italian pasta")

'To make Italian pasta, you will need the following ingredients:\n\n- 2 cups of all-purpose flour\n- 2 large eggs\n- 1/2 teaspoon of salt\n- Water (if needed)\n\nHere is a step-by-step guide on how to make Italian pasta:\n\n1. On a clean work surface, pour the flour and make a well in the center.\n2. Crack the eggs into the well and add the salt.\n3. Using a fork, gradually mix the eggs into the flour until a dough forms.\n4. Knead the dough for about 10 minutes until it becomes smooth and elastic. If the dough is too dry, add a little water. If it is too wet, add a little more flour.\n5. Wrap the dough in plastic wrap and let it rest for at least 30 minutes.\n6. After resting, divide the dough into smaller portions and roll each portion out into a thin sheet using a pasta machine or a rolling pin.\n7. Cut the pasta sheet into your desired shape, such as fettuccine or spaghetti.\n8. Cook the pasta in a large pot of boiling salted water for about 2-3 minutes or until al dente.\n9. Drain

In [15]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.globals import set_llm_cache, set_debug
from langchain.cache import InMemoryCache

set_llm_cache(InMemoryCache())
set_debug(True)

chat = ChatOpenAI(
    temperature=0.1,
    # streaming=True,
    # callbacks=[
    #     StreamingStdOutCallbackHandler(),
    # ],
)

chat.predict("How do you make italian pasta")

[llm/start] [1:llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: How do you make italian pasta"
  ]
}
[llm/end] [1:llm:ChatOpenAI] [7.36s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "To make Italian pasta, you will need the following ingredients:\n\n- 2 cups of all-purpose flour\n- 2 large eggs\n- Pinch of salt\n- Water (if needed)\n\nHere is a step-by-step guide to making Italian pasta:\n\n1. On a clean work surface, pour the flour and make a well in the center.\n2. Crack the eggs into the well and add a pinch of salt.\n3. Using a fork, gradually mix the eggs into the flour until a dough forms.\n4. Knead the dough for about 10 minutes until it is smooth and elastic. If the dough is too dry, add a little water. If it is too wet, add a little more flour.\n5. Wrap the dough in plastic wrap and let it rest for at least 30 minutes.\n6. After resting, roll out the dough using a pasta machine or a rolling pin until it is thin.\n7. 

'To make Italian pasta, you will need the following ingredients:\n\n- 2 cups of all-purpose flour\n- 2 large eggs\n- Pinch of salt\n- Water (if needed)\n\nHere is a step-by-step guide to making Italian pasta:\n\n1. On a clean work surface, pour the flour and make a well in the center.\n2. Crack the eggs into the well and add a pinch of salt.\n3. Using a fork, gradually mix the eggs into the flour until a dough forms.\n4. Knead the dough for about 10 minutes until it is smooth and elastic. If the dough is too dry, add a little water. If it is too wet, add a little more flour.\n5. Wrap the dough in plastic wrap and let it rest for at least 30 minutes.\n6. After resting, roll out the dough using a pasta machine or a rolling pin until it is thin.\n7. Cut the dough into your desired shape, such as fettuccine or spaghetti.\n8. Cook the pasta in a large pot of boiling salted water for 2-3 minutes or until al dente.\n9. Drain the pasta and toss it with your favorite sauce or toppings.\n\nEnjoy

In [16]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.globals import set_llm_cache, set_debug
from langchain.cache import InMemoryCache, SQLiteCache

set_llm_cache(SQLiteCache("cache.db"))


chat = ChatOpenAI(
    temperature=0.1,
    # streaming=True,
    # callbacks=[
    #     StreamingStdOutCallbackHandler(),
    # ],
)

chat.predict("How do you make italian pasta")

[llm/start] [1:llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: How do you make italian pasta"
  ]
}
[llm/end] [1:llm:ChatOpenAI] [6.18s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "To make Italian pasta, you will need the following ingredients:\n\n- 2 cups of all-purpose flour\n- 2 large eggs\n- 1/2 teaspoon of salt\n- Water (if needed)\n\nHere is a step-by-step guide to making Italian pasta:\n\n1. On a clean work surface, pour the flour and make a well in the center.\n2. Crack the eggs into the well and add the salt.\n3. Using a fork, slowly incorporate the flour into the eggs until a dough forms.\n4. Knead the dough for about 10 minutes until it is smooth and elastic. If the dough is too dry, add a little water. If it is too wet, add a little more flour.\n5. Wrap the dough in plastic wrap and let it rest for at least 30 minutes.\n6. After resting, roll out the dough using a pasta machine or a rolling pin until it is thin.

'To make Italian pasta, you will need the following ingredients:\n\n- 2 cups of all-purpose flour\n- 2 large eggs\n- 1/2 teaspoon of salt\n- Water (if needed)\n\nHere is a step-by-step guide to making Italian pasta:\n\n1. On a clean work surface, pour the flour and make a well in the center.\n2. Crack the eggs into the well and add the salt.\n3. Using a fork, slowly incorporate the flour into the eggs until a dough forms.\n4. Knead the dough for about 10 minutes until it is smooth and elastic. If the dough is too dry, add a little water. If it is too wet, add a little more flour.\n5. Wrap the dough in plastic wrap and let it rest for at least 30 minutes.\n6. After resting, roll out the dough using a pasta machine or a rolling pin until it is thin.\n7. Cut the dough into your desired shape, such as fettuccine or spaghetti.\n8. Cook the pasta in a large pot of salted boiling water for 2-3 minutes, or until al dente.\n9. Drain the pasta and toss it with your favorite sauce or toppings.\n\

# Serialization

In [17]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import get_openai_callback

chat = ChatOpenAI(
    temperature=0.1,
)


with get_openai_callback() as usage:
    a = chat.predict("What is the recipe for soju")
    b = chat.predict("What is the recipe for bread")
    print(a, "\n")
    print(b, "\n")
    print(usage)

[llm/start] [1:llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: What is the recipe for soju"
  ]
}
[llm/end] [1:llm:ChatOpenAI] [3.84s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "Ingredients:\n- 1 cup of rice\n- 1 cup of water\n- 1 tablespoon of nuruk (fermentation starter)\n- 1 tablespoon of yeast\n\nInstructions:\n1. Rinse the rice thoroughly and soak it in water for at least 1 hour.\n2. Drain the rice and steam it until fully cooked.\n3. Let the rice cool down to room temperature.\n4. In a large bowl, mix the nuruk and yeast with water until dissolved.\n5. Add the cooked rice to the bowl and mix well.\n6. Cover the bowl with a clean cloth and let it ferment in a warm place for 3-4 days.\n7. After fermentation, strain the mixture through a cheesecloth to remove any solids.\n8. Transfer the liquid to a clean container and let it sit for another 1-2 days to allow the flavors to develop.\n9. Your homemade soju is now ready t

In [18]:
from langchain.chat_models import ChatOpenAI
from langchain.llms.openai import OpenAI

chat = OpenAI(
    temperature=0.1,
    max_tokens=100,
    model="gpt-3.5-turbo" # 원하는 모델 입력
)

chat.save("model.json")

In [19]:
from langchain.chat_models import ChatOpenAI
from langchain.llms.openai import OpenAI
from langchain.llms.loading import load_llm

chat = load_llm("./model.json")

chat

/opt/homebrew/Caskroom/miniforge/base/envs/fullstack-gpt/lib/python3.11/site-packages/langchain/llms/openai.py:216: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(
/opt/homebrew/Caskroom/miniforge/base/envs/fullstack-gpt/lib/python3.11/site-packages/langchain/llms/openai.py:811: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(


OpenAIChat(client=<class 'openai.api_resources.chat_completion.ChatCompletion'>, model_kwargs={'temperature': 0.1, 'max_tokens': 100, 'top_p': 1, 'frequency_penalty': 0, 'presence_penalty': 0, 'n': 1, 'request_timeout': None, 'logit_bias': {}})